# This notebook shows how to estimate a spot with a gaussian model.
* It is possible to fit a spot with one or several gaussians.
* Differents algorithmes are available.

In [ ]:
%matplotlib notebook

import multiprocessing.pool
import pathlib

import matplotlib.pyplot as plt
import torch
import tqdm

from laueimproc import Diagram
from laueimproc.io.download import get_samples  # gives access to the dataset

### Firstly, we have to create the diagrams from images
* To see all the ways to create a diagram, please have a look to the `init_diagram` notebook.
* Take care having enough RAM and swappiness to load all the images, otherwise it would bias the peak search time.

In [ ]:
all_files = sorted(get_samples().iterdir())  # the list of all images path
diagrams = [Diagram(f) for f in all_files]
for diagram in diagrams:
    diagram.find_spots()

## Fit by one gaussian

In [ ]:
# very fast fit with "expectation maximization" algorithm
mean_em, cov_em, infodict_em = diagrams[0].fit_gaussian_em()  # like pca and barycenter
print("shape mean:", mean_em.shape)  # the position i, j of the center of the gaussian for each spot
print("shape cov:", cov_em.shape)  # the covariance matrix of the gaussian for each spot
assert infodict_em == {}  # empty because no parameters provided
*_, infodict_em = diagrams[0].fit_gaussian_em(aic=True, bic=True, eigtheta=True, log_likelihood=True)
print(sorted(infodict_em))  # more informations
*_, infodict_em = diagrams[0].fit_gaussian_em(photon_density=10900.0, tol=True)
print(sorted(infodict_em))  # mean standard deviation estimator

# fast fit with classical loss minimization
mean_mse, cov_mse, mag_mse, infodict_mse = diagrams[0].fit_gaussian(loss="mse", parallel=False)
print("shape mean:", mean_em.shape)  # the position i, j of the center of the gaussian for each spot
print("shape cov:", cov_em.shape)  # the covariance matrix of the gaussian for each spot